In [8]:
import json

data = []
# Open the file and iterate over each line
with open('fine-tuning-dataset/finanzas_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        # Load each non-empty line as a separate JSON object
        if line.strip():
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line.strip()}. Error: {e}")
                
# 'data' is now a list of all the JSON objects from the file
print(f"Successfully loaded {len(data)} objects.")


Successfully loaded 770 objects.


In [7]:
!pip install unsloth trl peft accelerate bitsandbytes

  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.9.0-cp313-cp313-win_amd64.whl.metadata (30 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
   ---------------------------------------- 0.0/556.4 kB ? eta -:--:--
   ---------------------------------------- 556.4/556.4 kB 18.3 MB/s  0:00:00
   ---------------------------------------- 0.0/59.0 MB ? eta -:--:--
   ---- ----------------------------------- 6.0/59.0 MB 28.6 MB/s eta 0:00:02
   -------- ------------------------------- 12.1/59.0 MB 29.1 MB/s eta 0:00:02
   ------------ --------------------------- 17.8/59.0 MB 28.3 MB/s eta 0:00:02
   ---------------- ----------------------- 24.1/59.0 MB 29.0 MB/s eta 0:00:02
   -------------------- ------------------- 30.1/59.0 MB 29.0 MB/s eta 0:00:01
   ------------------------- -------------- 37.5/

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [9]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

NotImplementedError: Unsloth cannot find any torch accelerator? You need a GPU.

In [ ]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
    ),
)

In [ ]:
# Train the model
trainer_stats = trainer.train()

In [ ]:
#Puede tardar 20 mins, es para exportar el modelo con formato legible para Ollama
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)